In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import deepchem as dc
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers

2021-08-30 17:11:03.889209: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/opt/intel/parallel_studio_xe_2019.0.045/clck_2019/lib:/opt/intel/parallel_studio_xe_2019.0.045/itac_2019/l

In [3]:
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import random
import tqdm
import copy
#torch.multiprocessing.set_sharing_strategy('file_system') # this is important
# ulimit -n 500000
#def set_worker_sharing_strategy(worker_id: int) -> None:
#    torch.multiprocessing.set_sharing_strategy('file_system')

In [4]:
import sys
sys.path = ['/home/chrisw/Documents/projects/2021/graph-transformer/src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeSeq2SeqFC, Deepchem2TorchGeometric, collate_minc_rndc_y, DFSCodeSeq2SeqFCLegacy

In [5]:
wandb.init(project='moleculenet10', entity='chrisxx')

config = wandb.config
config.n_atoms = 118
config.n_bonds = 4
config.emb_dim = 120
config.nhead = 12
config.nlayers = 6
config.use_Hs = True
config.max_nodes = 400
config.max_edges = 600
config.max_nodes_data = 400
config.max_edges_data = 600
config.onlyRandom = True
config.use_min = False # interestingly random seems to work better than using the minimal codes...
config.dim_feedforward = 2048
config.lr = 0.00003 # 10x smaller than the learning rate of the pretraining seems to be good
config.clip_gradient = 0.5
config.n_epochs = 25
config.patience = 3
config.factor = 0.95
config.minimal_lr = 6e-8
config.batch_size = 64
config.valid_patience = 5
config.valid_minimal_improvement = 0.00
#config.pretrained_dir = '../../models/chembl/better_transformer/medium/neu/'
config.pretrained_dir = '../../models/chemblH/better_transformer/medium/'
config.num_workers = 0
config.load_last = True
config.dataset = 'clintox' # supported 'bbbp', 'clintox', 'hiv', 'tox21'
config.model_dir = '../../models/moleculenetH/%s/better_transformer/medium/'%config.dataset
config.seed = 123

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-08-30 17:11:09.109815: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/

In [6]:
random.seed(config.seed)
torch.manual_seed(config.seed)
np.random.seed(config.seed)

In [7]:
os.makedirs(config.model_dir, exist_ok=True)

In [8]:
ngpu=1
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

In [9]:
to_cuda = lambda T: [t.cuda() for t in T]

In [10]:
def get_random_scaffold_split(dataset):
    splitter = dc.splits.ScaffoldSplitter()
    shuffled = dataset.complete_shuffle()
    trainidx, valididx, testidx = splitter.split(shuffled)
    train = shuffled.select(trainidx)
    valid = shuffled.select(valididx)
    test = shuffled.select(testidx)
    return train, valid, test

In [11]:
if config.dataset == 'clintox':
    tasks, datasets, transformers = dc.molnet.load_clintox(reload=False, featurizer=dc.feat.RawFeaturizer(True), splitter=None)
elif config.dataset == 'tox21':
    tasks, datasets, transformers = dc.molnet.load_tox21(reload=False, featurizer=dc.feat.RawFeaturizer(True), splitter=None)
elif config.dataset == 'hiv':
    tasks, datasets, transformers = dc.molnet.load_hiv(reload=False, featurizer=dc.feat.RawFeaturizer(True), splitter=None)
elif config.dataset == 'bbbp':
    tasks, datasets, transformers = dc.molnet.load_bbbp(reload=False, featurizer=dc.feat.RawFeaturizer(True), splitter=None)

Failed to featurize datapoint 7, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
Failed to featurize datapoint 302, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
Failed to featurize datapoint 983, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
Failed to featurize datapoint 984, None. Appending empty array
Exception messa

In [12]:
from sklearn.metrics import roc_auc_score, average_precision_score

def score(loader, model, model_head, use_min=False):
    val_roc = 0
    with torch.no_grad():
        full_preds, target = [], []
        for batch in tqdm.tqdm(loader):
            rndc, minc, z, eattr, y = batch
            if use_min:
                code = to_cuda(minc)
            else:
                code = to_cuda(rndc)
            y = y.to(device)
            self_attn, features, eos = model.encode(code, to_cuda(z), to_cuda(eattr)) # not clear whether to use minc or randc
            pred = sigmoid(model_head(features)).squeeze()
            
            pred_np = pred.detach().cpu().numpy().tolist()
            y_np = y.detach().cpu().numpy().tolist()
            full_preds += pred_np
            target += y_np

        target = np.asarray(target)
        full_preds = np.asarray(full_preds)
        roc = roc_auc_score(target, full_preds)
        prc = average_precision_score(target, full_preds)
    return roc, prc

In [ ]:
roc_avgs = []
prc_avgs = []
for rep in range(10):
    model = DFSCodeSeq2SeqFC(n_atoms=config.n_atoms,
                         n_bonds=config.n_bonds, 
                         emb_dim=config.emb_dim, 
                         nhead=config.nhead, 
                         nlayers=config.nlayers, 
                         max_nodes=config.max_nodes, 
                         max_edges=config.max_edges,
                         atom_encoder=nn.Embedding(config.n_atoms, config.emb_dim), 
                         bond_encoder=nn.Linear(config.n_bonds, config.emb_dim))
    if config.load_last:
        model.load_state_dict(torch.load(config.pretrained_dir+'checkpoint.pt', map_location=device))
    model_head = nn.Linear(5*config.emb_dim, 1)
    model_head.to(device)
    model.to(device)
    
    trainset, validset, testset = get_random_scaffold_split(datasets[0])
    collate_fn = collate_minc_rndc_y
    traindata = Deepchem2TorchGeometric(trainset, -1, useHs=config.use_Hs, onlyRandom=config.onlyRandom, max_nodes=config.max_nodes_data, max_edges=config.max_edges_data)
    validdata = Deepchem2TorchGeometric(validset, -1, useHs=config.use_Hs, onlyRandom=config.onlyRandom, max_nodes=config.max_nodes_data, max_edges=config.max_edges_data)
    testdata = Deepchem2TorchGeometric(testset, -1, useHs=config.use_Hs, onlyRandom=config.onlyRandom, max_nodes=config.max_nodes_data, max_edges=config.max_edges_data)
    trainloader = DataLoader(traindata, batch_size=config.batch_size, shuffle=True, pin_memory=False, collate_fn=collate_fn, num_workers=config.num_workers)
    validloader = DataLoader(validdata, batch_size=config.batch_size, shuffle=False, pin_memory=False, collate_fn=collate_fn, num_workers=config.num_workers)
    testloader = DataLoader(testdata, batch_size=config.batch_size, shuffle=False, pin_memory=False, collate_fn=collate_fn, num_workers=config.num_workers)
    
    optim = optimizers.Adam(list(model.parameters()) + list(model_head.parameters()), lr=config.lr)
    lr_scheduler = optimizers.lr_scheduler.ReduceLROnPlateau(optim, mode='min', verbose=True, patience=config.patience, factor=config.factor)
    early_stopping_model = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                                  path=config.model_dir+'checkpoint_model.pt')
    early_stopping_head = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                                  path=config.model_dir+'checkpoint_head.pt')
    bce = torch.nn.BCEWithLogitsLoss()
    sigmoid = nn.Sigmoid()
    
    valid_scores = []
    try:
        for epoch in range(config.n_epochs):  
            epoch_loss = 0
            pbar = tqdm.tqdm(trainloader)
            model.train()
            for i, data in enumerate(pbar):
                optim.zero_grad()
                rndc, minc, z, eattr, y = data
                if config.use_min:
                    code = to_cuda(minc)
                else:
                    code = to_cuda(rndc)
                z = to_cuda(z)
                eattr = to_cuda(eattr)
                y = y.to(device)
                #prediction
                self_attn, features, eos = model.encode(code, z, eattr)
                prediction = model_head(features).squeeze()
                loss = bce(prediction, y)

                loss.backward()
                if config.clip_gradient is not None:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.clip_gradient)
                optim.step()
                epoch_loss = (epoch_loss*i + loss.item())/(i+1)
                pbar.set_description('Epoch %d: CE %2.6f'%(epoch+1, epoch_loss))
            model.eval()
            roc_auc_valid, prc_auc_valid = score(validloader, model, model_head, config.use_min) 
            valid_scores += [roc_auc_valid]
            lr_scheduler.step(epoch_loss)
            early_stopping_model(-roc_auc_valid, model)
            early_stopping_head(-roc_auc_valid, model_head)
            curr_lr = list(optim.param_groups)[0]['lr']
            wandb.log({'loss':epoch_loss, 'roc_valid':roc_auc_valid, 'prc_valid':prc_auc_valid, 'learning rate':curr_lr}, 
                      step=config.n_epochs*rep + epoch)
            print('ROCAUC',roc_auc_valid, 'PRCAUC', prc_auc_valid)

            if early_stopping_model.early_stop:
                break

            if curr_lr < config.minimal_lr:
                break

    except KeyboardInterrupt:
        torch.save(model.state_dict(), config.model_dir+'_keyboardinterrupt.pt')
        print('keyboard interrupt caught')
        
    model.load_state_dict(torch.load(config.model_dir+'checkpoint_model.pt'))
    model_head.load_state_dict(torch.load(config.model_dir+'checkpoint_head.pt'))
    model.eval()
    roc_auc_valid, prc_auc_valid = score(testloader, model, model_head) 
    wandb.log({'roc_test':roc_auc_valid, 'prc_test':prc_auc_valid}, step=config.n_epochs*(rep+1))
    
    roc, prc = score(testloader, model, model_head, True)
    wandb.log({'roc_test_min':roc, 'prc_test_min':prc}, step=config.n_epochs*(rep+1))
    print('ROC, PRC VALID', score(validloader, model, model_head, True))
    print('ROC, PRC TEST', score(testloader, model, model_head, True))
    
    avg_roc = 0
    avg_prc = 0
    for i in range(20):
        roc, prc = score(testloader, model, model_head, False)
        avg_roc += roc
        avg_prc += prc
    avg_roc /= 20
    avg_prc /= 20 
    roc_avgs += [avg_roc]
    prc_avgs += [avg_prc]
    wandb.log({'roc_test_avg20':avg_roc, 'prc_test_avg20':avg_prc}, step=config.n_epochs*(rep+1))
    print('ROC, PRC TEST', avg_roc, avg_prc)
    del model
    del model_head
wandb.log({'roc_test_avgavg':np.mean(roc_avgs), 'prc_test_avgavg':np.mean(prc_avgs)}, step=config.n_epochs*(rep+1))
wandb.log({'roc_test_avgstd':np.std(roc_avgs), 'prc_test_avgstd':np.std(prc_avgs)}, step=config.n_epochs*(rep+1))
wandb.run.summary["roc_test_mean"] = np.mean(roc_avgs) 
wandb.run.summary["roc_test_std"] = np.std(roc_avgs)
wandb.run.summary["prc_test_mean"] = np.mean(prc_avgs)
wandb.run.summary["prc_test_std"] = np.std(prc_avgs)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10.47it/s]


ROCAUC 0.5031339031339032 PRCAUC 0.08943299490683802


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10.46it/s]


ROCAUC 0.5481481481481482 PRCAUC 0.10255779207254752


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10.74it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  8.15it/s]


EarlyStopping counter: 3 out of 5
EarlyStopping counter: 3 out of 5
ROCAUC 0.6562749800159873 PRCAUC 0.09700275206737738


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  8.37it/s]


ROCAUC 0.7490007993605117 PRCAUC 0.13238737243784107


Epoch 9: CE 0.174309:  73%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                         | 27/37 [00:06<00:02,  4.02it/s]